# JapDic 
Version 2.0
Made by Vik

In [25]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from notion_client import Client
import os
from dotenv import load_dotenv

In [26]:
load_dotenv("locals.env")
token = os.getenv("NOTION_TOKEN")
table_id = os.getenv("NOTION_TABLE_ID")
page_id = os.getenv("NOTION_JAPDIC_DEMO_PAGE_ID")


In [27]:
# Read the words from the text file
with open("data/18_11_list.txt", "r") as f:
    for line in f:
        words = line.strip().split(",")
        english_word = words[0]
        romaji_word = words[1]
        page_title = f"{english_word} - {romaji_word}"

In [28]:
# database_id = os.getenv("NOTION_DATABASE_ID")

client = Client(auth=token)

print(page_id)

142ebe12-09f2-807c-aa24-e597aa2ec4de


In [29]:
page = client.pages.retrieve(page_id=page_id)

# print(page)

## Add header case

In [30]:
header_block = {
    "object": "block",
    "type": "header",
    "header": {"text": [{"text": {"content": "This is a header"}}]},
}

# response = client.pages.update(page_id=page_id, children=[header_block])
# print(response)

In [31]:
# Function to add text to a page
def add_text_to_page(page_id: str, text: str):
    """
    Adds text as a paragraph block to a Notion page

    Args:
        page_id (str): The ID of the Notion page
        text (str): The text content to add
    """
    try:
        response = client.blocks.children.append(
            block_id=page_id,
            children=[
                {
                    "type": "paragraph",
                    "paragraph": {
                        "rich_text": [
                            {"type": "text", "text": {"content": text, "link": None}}
                        ],
                        "color": "default",
                    },
                }
            ],
        )
        return response
    except Exception as e:
        print(f"Error adding text to page: {e}")
        return None


# Example usage
# page_id = "YOUR_PAGE_ID"  # Replace with your page ID
add_text_to_page(page_id, "demo text")

{'object': 'list',
 'results': [{'object': 'block',
   'id': '143ebe12-09f2-817c-a9a9-f0cd0b8d7248',
   'parent': {'type': 'page_id',
    'page_id': '142ebe12-09f2-807c-aa24-e597aa2ec4de'},
   'created_time': '2024-11-19T14:45:00.000Z',
   'last_edited_time': '2024-11-19T14:45:00.000Z',
   'created_by': {'object': 'user',
    'id': 'b323a920-ffe3-4bd1-b54c-51e9c8304629'},
   'last_edited_by': {'object': 'user',
    'id': 'b323a920-ffe3-4bd1-b54c-51e9c8304629'},
   'has_children': False,
   'archived': False,
   'in_trash': False,
   'type': 'paragraph',
   'paragraph': {'rich_text': [{'type': 'text',
      'text': {'content': 'demo text', 'link': None},
      'annotations': {'bold': False,
       'italic': False,
       'strikethrough': False,
       'underline': False,
       'code': False,
       'color': 'default'},
      'plain_text': 'demo text',
      'href': None}],
    'color': 'default'}}],
 'next_cursor': None,
 'has_more': False,
 'type': 'block',
 'block': {},
 'request_id'

## Table creation 

In [32]:
def create_japanese_dictionary_table(page_id: str):
    """
    Creates a database (table) with columns: ID, ROOMA-JI, Pronounced, English

    Args:
        page_id (str): The ID of the parent page where the database will be created
    """
    try:
        response = client.databases.create(
            parent={"type": "page_id", "page_id": page_id},
            title=[{"type": "text", "text": {"content": "Japanese Dictionary"}}],
            properties={
                "ID": {"type": "number", "number": {"format": "number"}},
                "ROOMA-JI": {
                    "type": "title",  # Using title type as it's required for at least one column
                    "title": {},
                },
                "Pronounced": {"type": "rich_text", "rich_text": {}},
                "English": {"type": "rich_text", "rich_text": {}},
            },
        )
        return response
    except Exception as e:
        print(f"Error creating database: {e}")
        return None

In [33]:
# Example usage
database = create_japanese_dictionary_table(page_id)

## Table

In [34]:
def check_for_existing_table(page_id: str) -> str | None:
    """
    Checks if a table already exists on the page

    Args:
        page_id (str): The ID of the Notion page

    Returns:
        str | None: Table ID if found, None otherwise
    """
    try:
        # Get all blocks on the page
        response = client.blocks.children.list(block_id=page_id)

        # Look for a table block
        for block in response["results"]:
            if block["type"] == "table":
                return block["id"]

        return None
    except Exception as e:
        print(f"Error checking for existing table: {e}")
        return None

In [35]:
def create_japanese_table(page_id: str):
    """
    Creates a table with columns: ID, ROOMA-JI, Pronounced, English

    Args:
        page_id (str): The ID of the parent page where the table will be created
    """
    try:
        response = client.blocks.children.append(
            block_id=page_id,
            children=[
                {
                    "type": "table",
                    "table": {
                        "table_width": 4,
                        "has_column_header": True,
                        "has_row_header": False,
                        "children": [
                            # Header row
                            {
                                "type": "table_row",
                                "table_row": {
                                    "cells": [
                                        [{"type": "text", "text": {"content": "ID"}}],
                                        [
                                            {
                                                "type": "text",
                                                "text": {"content": "ROOMA-JI"},
                                            }
                                        ],
                                        [
                                            {
                                                "type": "text",
                                                "text": {"content": "Pronounced"},
                                            }
                                        ],
                                        [
                                            {
                                                "type": "text",
                                                "text": {"content": "English"},
                                            }
                                        ],
                                    ]
                                },
                            }
                            # You can add more rows here if needed
                        ],
                    },
                }
            ],
        )
        return response
    except Exception as e:
        print(f"Error creating table: {e}")
        return None


In [36]:
def get_or_create_table(page_id: str) -> str | None:
    """
    Gets existing table ID or creates new table

    Args:
        page_id (str): The ID of the Notion page

    Returns:
        str | None: Table ID if found/created, None if failed
    """
    # First check for existing table
    table_id = check_for_existing_table(page_id)

    if table_id:
        print("Found existing table")
        return table_id

    # If no table exists, create new one
    print("Creating new table")
    response = create_japanese_table(page_id)
    if response and "results" in response:
        table_id = response["results"][0]["id"]

        # Store the table ID in .env file
        with open("locals.env", "a") as f:
            f.write(f"\nNOTION_TABLE_ID={table_id}")

        return table_id

    return None

In [37]:
def add_table_row(table_id: str, id_num: int, rooma_ji: str, english: str):
    """
    Adds a new row to the table

    Args:
        table_id (str): The ID of the table
        id_num (int): The ID number
        rooma_ji (str): The romanized Japanese text
        english (str): The English translation
    """
    try:
        response = client.blocks.children.append(
            block_id=table_id,
            children=[
                {
                    "type": "table_row",
                    "table_row": {
                        "cells": [
                            [{"type": "text", "text": {"content": str(id_num)}}],
                            [{"type": "text", "text": {"content": rooma_ji}}],
                            [
                                {"type": "text", "text": {"content": ""}}
                            ],  # Empty Pronounced cell
                            [{"type": "text", "text": {"content": english}}],
                        ]
                    },
                }
            ],
        )
        return response
    except Exception as e:
        print(f"Error adding row: {e}")
        return None

In [40]:
# Example usage
# page_id = "YOUR_PAGE_ID"  # Replace with your page ID

# Get or create table if needed
if not table_id:
    table_id = get_or_create_table(page_id)
    print(table_id)
    if not table_id:
        print("Failed to get or create table")
        exit(1)

# Now you can use the table_id for adding rows
# add_table_row(table_id, 1, "konnichiwa", "hello")

## Tests

In [39]:
# Test function to verify word parsing
def test_parse_word_line():
    # Test case 1: Normal case
    test_line = "hello,konnichiwa"
    english, romaji = test_line.strip().split(",")
    assert english == "hello"
    assert romaji == "konnichiwa"

    # Test case 2: With spaces
    test_line = "good morning, ohayou gozaimas"
    english, romaji = test_line.strip().split(",")
    assert english == "good morning"
    assert romaji == " ohayou gozaimas"

    # Test case 3: Empty values
    test_line = ","
    english, romaji = test_line.strip().split(",")
    assert english == ""
    assert romaji == ""


# Run the test
test_parse_word_line()
print("All tests passed!")


All tests passed!
